In [1]:
from scipy import optimize
import numpy as np
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('pulsar_star_dataset.csv')
df.head()

,MoIP,SDoIP,EkIP,SoIP,MoDM-SNR,SDDM-SNR,EkDM-SNR,SoDM-SNR,Class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [3]:
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train

,MoIP,SDoIP,EkIP,SoIP,MoDM-SNR,SDDM-SNR,EkDM-SNR,SoDM-SNR
14453,108.765625,47.520470,0.191285,0.007945,2.921405,18.009581,8.468258,84.137111
15797,121.554688,50.909357,0.083099,-0.087226,1.098662,11.590431,14.180137,242.867752
4510,100.265625,43.808975,0.233088,0.451613,1.142977,13.774721,14.148388,215.404864
11899,118.460938,41.178518,0.162482,1.050363,1.161371,13.353561,14.472023,233.870475
895,97.109375,35.920319,0.364792,2.250635,2.826923,17.559245,8.733436,90.352868
...,...,...,...,...,...,...,...,...
11284,124.312500,53.179053,-0.012418,-0.556021,7.186455,29.308266,4.531382,21.725143
11964,115.617188,46.784600,0.218177,0.226757,6.140468,30.271961,5.732201,34.357282
5390,116.031250,43.213846,0.663456,0.433088,0.785117,11.628149,17.055215,312.204325
860,135.664062,49.933749,-0.089940,-0.226726,3.859532,21.501505,7.398395,62.334018


In [5]:
y_train

14453    0
15797    0
4510     0
11899    0
895      0
        ..
11284    0
11964    0
5390     0
860      0
15795    0
Name: Class, Length: 14318, dtype: int64

In [4]:
mean_train = X_train.mean()
std_train = X_train.std()
X_train_normalized = (X_train - mean_train) / std_train
X_train_normalized

,MoIP,SDoIP,EkIP,SoIP,MoDM-SNR,SDDM-SNR,EkDM-SNR,SoDM-SNR
14453,-0.090329,0.143529,-0.269773,-0.285864,-0.328411,-0.423906,0.031082,-0.199048
15797,0.408394,0.639497,-0.371681,-0.301340,-0.389983,-0.753052,1.291614,1.272535
4510,-0.421797,-0.399652,-0.230397,-0.213717,-0.388486,-0.641051,1.284608,1.017928
11899,0.287750,-0.784623,-0.296905,-0.116353,-0.387864,-0.662646,1.356029,1.189122
895,-0.544878,-1.554167,-0.106335,0.078828,-0.331602,-0.446997,0.089603,-0.141422
...,...,...,...,...,...,...,...,...
11284,0.515939,0.971670,-0.461654,-0.377572,-0.184338,0.155441,-0.837732,-0.777666
11964,0.176855,0.035834,-0.244442,-0.250282,-0.219671,0.204855,-0.572728,-0.660554
5390,0.193002,-0.486750,0.174996,-0.216730,-0.400574,-0.751118,1.926104,1.915350
860,0.958606,0.496716,-0.534678,-0.324024,-0.296721,-0.244855,-0.205022,-0.401183


In [7]:
# from sklearn import svm
# clf = svm.SVC(kernel='linear', C=1.0)
# clf.fit(X_train_normalized, y_train)

In [8]:
# X_test_normalized = (X_test - mean_train) / std_train
# y_pred = clf.predict(X_test_normalized)
# accuracy_score(y_test, y_pred)

In [5]:
X_train_normalized = np.array(X_train_normalized)
y_train = np.array(y_train)
X_test_normalized = (X_test - mean_train) / std_train
X_test_normalized = np.array(X_test_normalized)
y_test = np.array(y_test)

In [9]:
C = 1
m,n = X_train_normalized.shape
y_train = np.array(y_train)
X_train_normalized = np.array(X_train_normalized)
y_train = y_train.reshape(-1,1) * 1.
X_dash = y_train * X_train_normalized
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y_train.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#==================Computing and printing parameters===============================#


     pcost       dcost       gap    pres   dres
 0: -1.3000e+04 -2.7318e+04  6e+04  1e+00  3e-16
 1: -1.0801e+04 -1.8017e+04  7e+03  1e-02  2e-16
 2: -1.2764e+04 -1.3061e+04  3e+02  6e-04  4e-16
 3: -1.2998e+04 -1.3001e+04  3e+00  6e-06  2e-16
 4: -1.3000e+04 -1.3000e+04  3e-02  6e-08  2e-16
 5: -1.3000e+04 -1.3000e+04  3e-04  6e-10  1e-16
Optimal solution found.


IndexError: Boolean index has wrong length: 14318 instead of 17898

In [10]:
w = ((y_train * alphas).T @ X_train_normalized).reshape(-1,1)
S = (alphas > 1e-4).flatten()
b = y_train[S] - np.dot(X_train_normalized[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

Alphas =  [0.99999998 0.99999998 0.99999998 ... 0.99999998 0.99999998 0.99999998]
w =  [-6.92382594e-22 -1.06824745e-21  1.19044592e-21  1.21720188e-21
 -3.06659899e-23  5.81637137e-22 -2.83751437e-22 -9.30735478e-23]
b =  [9.86668259e-22]


In [11]:
X_test_normalized = (X_test - mean_train) / std_train
X_test_normalized = np.array(X_test_normalized)
y_pred = np.sign(np.dot(X_test_normalized, w) + b[0])
accuracy_score(y_test, y_pred)
# # X_test_normalized.shape

0.08743016759776537

In [6]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
             
def linear_kernel(x1, x2):
    return np.dot(x1, x2) * 1.

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

class SVM(object):

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples)) * 1.
        b = cvxopt.matrix(0.0) * 1.

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # A = A.astype('d')
        # b = b.astype('d')
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))




In [7]:
X_train_normalized = np.array(X_train_normalized)
y_train = np.array(y_train)
X_test_normalized = np.array(X_test_normalized)
y_test = np.array(y_test)
clf2 = SVM(kernel=linear_kernel, C=1.0)
clf2.fit(X_train_normalized, y_train)
y_pred = clf2.predict(X_test_normalized)
accuracy_score(y_test, y_pred)

     pcost       dcost       gap    pres   dres
 0: -1.3000e+04 -2.7318e+04  6e+04  1e+00  3e-16
 1: -1.0801e+04 -1.8017e+04  7e+03  1e-02  2e-16
 2: -1.2764e+04 -1.3061e+04  3e+02  6e-04  2e-16
 3: -1.2998e+04 -1.3001e+04  3e+00  6e-06  2e-16
 4: -1.3000e+04 -1.3000e+04  3e-02  6e-08  2e-16
 5: -1.3000e+04 -1.3000e+04  3e-04  6e-10  2e-16
Optimal solution found.
13000 support vectors out of 14318 points


0.9103351955307263

In [8]:
accuracy_score(y_test, y_pred)

0.9103351955307263

In [ ]:
import numpy as np
from cvxopt import matrix, spmatrix
from cvxopt.solvers import qp
from cvxopt import solvers

class LinearSVM():
	def __init__(self, C):
		self.C = C

	def fit(self, data, labels):
		K = data * labels[:, np.newaxis]
		K = np.dot(K, K.T)
		N = len(data)

		solvers.options['show_progress'] = False
		sol = qp(matrix(K), matrix(-np.ones((N, 1))), matrix(np.vstack((np.eye(N), -np.eye(N)))), 
			matrix(np.vstack((np.ones((N, 1)) * self.C, np.zeros((N, 1))))), 
			matrix(labels.reshape(1, N).astype('d')), matrix(np.zeros(1).astype('d')))

		self.alpha = np.array(sol['x']).reshape(N)

		self.support_ = [i for i in range(N) if self.alpha[i] > 1e-3]
		self.w = (data * (self.alpha * labels)[:, np.newaxis]).sum(axis=0)
		for i in range(N):
			if 0 < self.alpha[i] < self.C:
				self.bias = labels[i] - np.dot(self.w, data[i])
				break

	def predict(self, data):
		if len(data.shape) <= 1:
			self.predict(data.reshape(1, data.shape[0]))

		return np.sign(np.dot(data, self.w) + self.bias)

	def decision_function(self, data):
		return (np.dot(data, self.w) + self.bias) / np.linalg.norm(self.w)

	def score(self, data, labels):
		pr = self.predict(data)
		correct = 0.
		N = len(data)
		for i in range(N):
			correct += 1 if pr[i] * labels[i] > 0 else 0
		return correct / N

In [ ]:
clf3 = LinearSVM(C=1.0)
clf3.fit(X_train_normalized, y_train)
y_pred = clf3.predict(X_test_normalized)
clf3.score(X_test_normalized, y_test)

: 

: 